# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [2]:
# Import libraries
import pandas as pd
import numpy as np

# load the three datasets
url1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
url2 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv'
url3 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv'

df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df3 = pd.read_csv(url3)

print("File 1 shape:", df1.shape)
print("File 2 shape:", df2.shape)
print("File 3 shape:", df3.shape)

# Clean each dataframe separately before combining
def clean_dataframe(df):
    # Clean column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df = df.rename(columns={'st': 'state'})
    
    # Clean gender
    if 'gender' in df.columns:
        df['gender'] = df['gender'].replace(['Female', 'female', 'Femal'], 'F')
        df['gender'] = df['gender'].replace(['Male', 'male'], 'M')
    
    # Clean state
    if 'state' in df.columns:
        df['state'] = df['state'].replace({'AZ': 'Arizona', 'Cali': 'California', 'WA': 'Washington'})
    
    # Clean education
    if 'education' in df.columns:
        df['education'] = df['education'].replace('Bachelors', 'Bachelor')
    
    # Clean CLV
    if 'customer_lifetime_value' in df.columns:
        df['customer_lifetime_value'] = df['customer_lifetime_value'].astype(str).str.replace('%', '')
        df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
    
    # Clean vehicle class
    if 'vehicle_class' in df.columns:
        df['vehicle_class'] = df['vehicle_class'].replace(['Sports Car', 'Luxury SUV', 'Luxury Car'], 'Luxury')
    
    # Fix complaints
    if 'number_of_open_complaints' in df.columns:
        df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(
            lambda x: str(x).split('/')[1] if pd.notna(x) and '/' in str(x) else x
        )
        df['number_of_open_complaints'] = pd.to_numeric(df['number_of_open_complaints'], errors='coerce')
    
    return df

# Clean each file
df1_clean = clean_dataframe(df1)
df2_clean = clean_dataframe(df2)
df3_clean = clean_dataframe(df3)

print("\nCleaned individual files")

# Now combine them
combined_df = pd.concat([df1_clean, df2_clean, df3_clean], ignore_index=True)

print(f"Combined dataset shape: {combined_df.shape}")

# Handle null values
categorical_cols = ['gender', 'state', 'education', 'vehicle_class', 'response', 'coverage', 
                   'employmentstatus', 'location_code', 'marital_status', 'policy_type', 
                   'policy', 'renew_offer_type', 'sales_channel']

for col in categorical_cols:
    if col in combined_df.columns:
        mode_val = combined_df[col].mode()
        if len(mode_val) > 0:
            combined_df[col].fillna(mode_val[0], inplace=True)

numeric_cols = ['customer_lifetime_value', 'income', 'monthly_premium_auto', 
               'months_since_last_claim', 'months_since_policy_inception', 
               'number_of_open_complaints', 'number_of_policies', 'total_claim_amount']

for col in numeric_cols:
    if col in combined_df.columns:
        combined_df[col].fillna(combined_df[col].median(), inplace=True)

print("Null values handled")

# Convert to integers
int_cols = ['income', 'monthly_premium_auto', 'months_since_last_claim',
           'months_since_policy_inception', 'number_of_open_complaints', 'number_of_policies']

for col in int_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype(int)

print("Converted to integers")

# Remove duplicates
print(f"Duplicates before: {combined_df.duplicated().sum()}")

combined_df = combined_df.drop_duplicates(keep='first')
combined_df = combined_df.reset_index(drop=True)

print(f"Duplicates after: {combined_df.duplicated().sum()}")
print(f"\nFinal combined dataset shape: {combined_df.shape}")

# Save
combined_df.to_csv('combined_cleaned_data.csv', index=False)
print("Saved to combined_cleaned_data.csv")

File 1 shape: (4008, 11)
File 2 shape: (996, 11)
File 3 shape: (7070, 11)

Cleaned individual files
Combined dataset shape: (12074, 11)
Null values handled
Converted to integers
Duplicates before: 2939
Duplicates after: 0

Final combined dataset shape: (9135, 11)
Saved to combined_cleaned_data.csv


/var/folders/8w/jbgrr0fd4jvbmhsmhc4cb5400000gn/T/ipykernel_74894/3611547699.py:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df[col].fillna(mode_val[0], inplace=True)
/var/folders/8w/jbgrr0fd4jvbmhsmhc4cb5400000gn/T/ipykernel_74894/3611547699.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [ ]:
# Load the marketing customer dataset
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'
marketing_df = pd.read_csv(url)

print(f"Dataset shape: {marketing_df.shape}")
print(f"\nColumn names:")
print(marketing_df.columns)
marketing_df.head()

print(marketing_df.info())

# Create pivot table for total revenue by sales channel
revenue_by_channel = marketing_df.pivot_table(
    values='total_claim_amount',
    index='sales_channel',
    aggfunc='sum'
).round(2)

print("Total Revenue by Sales Channel:")
print(revenue_by_channel)

# Or sort it to see which channel brought the company the most revenue
revenue_sorted = revenue_by_channel.sort_values('total_claim_amount', ascending=False)
print("\nSales Channels ranked by revenue:")
print(revenue_sorted)

# insights
print("\nInsights:")
print(f"The sales channel with highest revenue is: {revenue_sorted.index[0]}")
print(f"Total revenue from {revenue_sorted.index[0]}: ${revenue_sorted.iloc[0]['total_claim_amount']:,.2f}")

# Part 2

# how to reate pivot table for avg customer lifetime value by gender and education
clv_pivot = marketing_df.pivot_table(
    values='customer_lifetime_value',
    index='gender',
    columns='education',
    aggfunc='mean'
).round(2)

print("Average Customer Lifetime Value by Gender and Education:")
print(clv_pivot)

# final step is to review the results
print("\nInsights:")
print(f"Highest avg CLV: {clv_pivot.max().max():.2f}")
print(f"Lowest avg CLV: {clv_pivot.min().min():.2f}")

# find which combination has the highest return
max_val = clv_pivot.max().max()
for col in clv_pivot.columns:
    for idx in clv_pivot.index:
        if clv_pivot.loc[idx, col] == max_val:
            print(f"Highest CLV is for {idx} with {col} education")

            

Dataset shape: (10910, 27)

Column names:
Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unnamed:_0                     10910 non-null  int64  
 1   customer                       10910 non-null  object 
 2   state                          10910 non-null  object

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [9]:
# First extract month from a date column
print(marketing_df.columns)

print(marketing_df['effective_to_date'].head())

marketing_df['effective_to_date'] = pd.to_datetime(marketing_df['effective_to_date'], errors='coerce')
marketing_df['month'] = marketing_df['effective_to_date'].dt.month

print(marketing_df[['effective_to_date', 'month']].head())

complaints_pivot = marketing_df.pivot_table(
    values='number_of_open_complaints',
    index='policy_type',
    columns='month',
    aggfunc='sum',
    fill_value=0
)

print("Complaints by Policy Type and Month (wide format):")
print(complaints_pivot)

complaints_long = complaints_pivot.reset_index()
complaints_long = complaints_long.melt(
    id_vars='policy_type',
    var_name='month',
    value_name='number_of_complaints'
)

# sort by number of complaints
complaints_long = complaints_long.sort_values('number_of_complaints', ascending=False)

print("\nComplaints by Policy Type and Month (long format):")
print(complaints_long)

# Show top 10 policy type/month combinations with most complaints
print("\nTop 10 Policy Type/Month combinations with most complaints:")
print(complaints_long.head(10))

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')
0    2011-02-18
1    2011-01-18
2    2011-02-10
3    2011-01-11
4    2011-01-17
Name: effective_to_date, dtype: object
  effective_to_date  month
0        2011-02-18      2
1        2011-01-18      1
2        2011-02-10      2
3        2011-01-11      1
4        2011-01-17      1
Complaints by Policy Type and Month (wide format):
month                     1            2
policy_type                             
Corporate Auto   443.434952   385.2